# Quickstart – gwrefpy

This example demonstrates how to use `gwrefpy` to analyze an observation and a reference well, calibrate a model, save the project, and then reload and visualize the results.

We will go step by step:
1. Store the data in `Well` objects  
2. Build a `Model` and run calibration  
3. Save the project and reload it  
4. Plot the results  


In [ ]:
import logging
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from gwrefpy import Model, Well

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.WARNING)

## 1. Import groundwaterlevel measurements 
Importing groundwaterlevel for a reference well and a observationswell as csv files and creates panda series.

In [ ]:
# Loading observation well data
obs_df = pd.read_csv("examples/data/obs_well.csv", parse_dates=["date"])
obs_series = pd.Series(obs_df["value"].astype(float).values, index=obs_df["date"])

# Loading reference well data
ref_df = pd.read_csv("examples/data/ref_well.csv", parse_dates=["date"])
ref_series = pd.Series(ref_df["value"].astype(float).values, index=ref_df["date"])


## 2. Create well objects

We now add our time series to `Well` objects.  
- `is_reference=False` for the observation well  
- `is_reference=True` for the reference well  

In [ ]:

obs = Well(name="12GIPGW", is_reference=False)
obs.add_timeseries(pd.Series(values_obs, index=dates))

ref = Well(name="45LOGW", is_reference=True)
ref.add_timeseries(pd.Series(values_ref, index=dates))

## 3. Build a model and calibrate

We create a `Model`, add both wells, and run calibration over the selected time window.

In [ ]:
model = Model(name="Strandangers")
model.add_well(obs)
model.add_well(ref)

calibration_period_start = dates[0]
calibration_period_end = dates[-1]

model.fit(
    obs,
    ref,
    time_equivalent="0D",
    tmin=calibration_period_start,
    tmax=calibration_period_end,
)

## 4. Save and reload the project

We save the model to a `.gwref` file and reload it again.  
This shows how projects can be shared and reused.

In [ ]:
project_path = "examples/strandangers_project"
model.save_project(project_path, overwrite=True)

# simulate a fresh session
del model, obs, ref

model_loaded = Model("Strandangers loaded")
model_loaded.open_project(f"{project_path}.gwref")

## 5. Plot the results

Finally, we plot the calibrated model using the built-in plotting functionality.

In [ ]:
model_loaded.plot(plot_style="fancy", color_style="monochrome")
plt.show()
